# RWKV

> 「変わることがなければ成長することもない  
> 成長することがなければ真に生きていない」  
> ビル・ゲイツ

## RWKVの概要

OpenAIが開発するChatGPTにおけるGPT4は強力であり、同様のモデルが様々登場しつつあるが、モデル動作に必要な計算資源も膨大であり、簡単に自宅やColabで動作させることはできないが、RWKVはこの問題を解決するのではないかと期待されている

言語モデルは、LSTMなどRNNベースのモデルが利用されてきたが、Transformerの登場により状況が一変した
- Transformerにより並列処理が可能となり実行速度が向上した
- しかしながら、必要となる計算資源が膨大になった


## RNNとTransformerの違いを再確認

Recurrent Neural Networks (RNN)とTransformerの違いを再確認する

ある文章があり、その単語のトークン列が$F[0], F[1], ... F[n]$とする

- Transformer
  - Attention Weightを用いて$F[0] ... F[n-1]$の単語の依存関係から$F[n]$の単語を生成する
  - 文章全体の状態を保持して学習し、残差接続(residual connection)、Dropout、LayerNormなどを利用することで、層数を増やしても安定して学習を進めることができる
  - Self-Attentionは離れた位置の依存関係を学習できるが、シーケンス内のすべての要素と他のすべての要素との依存関係を計算するため、計算量とメモリ使用量がシーケンス長(トークン数)の2乗つまり、O(n^2)で大きくなるという欠点がある
  - 一方で計算を並列化することができるため、計算進度を高めることができる

- RNN
  - $F[n-1]$の単語から$F[n]$の単語を生成というプロセスを繰り返して学習
  - 単一状態を保持し、これを繰り返し適用するため、離れた位置ほど単語の依存関係が失われていく
      - 計算を繰り返すため、勾配爆発や勾配消失を発生ひやすい
  - メモリと計算量は文章長に対して線形にスケールする
  - 並列化と拡張性の制限からtransformerと同等の性能を達成することが困難


# RWKV(Receptance Weighted Key Value)とは?

transformerの効率的な並列学習とRNNの効率的な推論の両方を兼ね備えたモデル

名前は、利用する4つのパラメータ$R, W, K, V$に由来する


## RWKVの特徴

次のように纏めることができる

- RNNベースのモデルを用いて、推論の高速化と省メモリ化を実現
  - 推論時のメモリ使用率は最も大きい14Bモデルでも3GB程度
- 数百億のパラメータまでスケールする(これが限界かどうかは不明であるが、スケール則は有効と思われる)、非Transformerアーキテクチャ
  - この規模はRNNベースのモデルでは達成できなかった
- 同一サイズのTransformerと同等の性能を発揮
- 学習時はTransformerと同様の動作であるため、並列化が容易
- 文章長が理論上無限となる(学習時の文章長に依存し、実際は1024程度)
- AttentionではなくRWKVモデルを利用する

RWKVモデルは https://github.com/BlinkDL/ において公開されており、RWKV1からRWKV4までの4つのモデルが存在

ここでは、RWKV2を用いるが、このモデルは推論にRNNモード、学習にTransformerモードを利用する

## TransformerとRWKVの違い

構造の違いを図を用いて表すと次の通り

- RWKVは、特徴的なTime-mixing blockとChannel-mixing blockが存在
- Transformerと異なり、encoder-decoderモデルではない
- 全体の構造はTransformerと類似する
  - Time-mixing blockとmultihead attentionというブロックが同様の配置・連結された構造を持つが中身が異なる

<img src="http://class.west.sd.keio.ac.jp/dataai/text/transformer-rwkv.png" width=700>


# RWKVの詳細

RWKVを特長づけるブロックとしてTime-mixing blockとChannel-mixing blockがある

RWKVという名称の由来にもなっている4つのパラメータはTime-mixingブロックとChannel-mixingブロックで使用され、次の意味を持つ
- R:過去の情報の受容度を表現するReceptanceベクトル
- W:位置の重み減衰ベクトル。訓練可能なモデルパラメータ
- K:一般的なAttention Mechanismと同様のK(Key)ベクトル
- V:一般的なAttention Mechanismと同様のV(Value)ベクトル


## TransformerからRWKVへ

### Transformer

$$Attention(Q,K,V)=softmax(\frac{QK^T}{\sqrt{d_k}})V$$

ここでQ、K、Vはクエリ(Query)、キー(Key)、値(Value)を示し、キーの次元数$d_k$，シーケンス長(トークン数)$N$、次元数$d_v$である

$F[t+1]$の予測に、$F[0]...F[t]$の文章と、現在の単語$x_t$と$F[t]$をそれぞれ比較して文脈全体の依存関係を考慮する
- 具体的には$Qx_t$と$K(F[0]...F[t])$の全ての単語の内積によりAttention Weightを求め、前の各状態$F[i]$と比較して類似度を求める

すべてのQueryベクトルとKeyベクトルの内積を計算するため、計算量が$n^2$となる


### An Attention Free Transformer (AFT)

Self-Attentionの代わりに普通の全結合層(Fully Connected layer)を利用したモデル
- 計算コストを大幅に削減
- 条件によって、Transformerと同等またはそれ以上のパフォーマンスを達成

AFTは次のように求めることができる

$$Attn^+(W,K,V)_t=\frac{\sum^t_{i=1}e^{\omega t,i+k_i}v_i}{\sum^t_{i=1}e^{\omega t,i+k_i}}$$

ここで$\omega_{t,i} \in R^{T\times T}$は、学習した関係における位置バイアスを表す

また、厳密ではないがRWKVを数的に表現すると、

$$v_{i+1} = sigmoid(Rx[t])\cdot \sum^t_{i=0}{Attn^+(W,K,V)_t}$$

となるため、一つ先($t+1$番目)の単語を予測する場合は、$t$番目の単語の予測に$sigmoid(R*F[t])$、$t～0$番目の単語の予測に$v_i$と$\omega_{t,i+k_i}$の2つを用いる

- なお、活性化関数は$sigmoid$である必要はないが、$sigmoid$の性能が高いという評価結果がある
  - この$sigmoid$の項は正規化せずreceptanceと呼ぶ
- MultiHeadAttentionは存在せず、各単語に$0～t$番目の単語との依存関係を求める必要ない
  - $e^{\omega t,i+k_i}v_i$の項が依存関係を表現する

この式により、RNNと同様再帰的に適用することで、$t-1$番目から、$t$番目を予測できる
- 一つ前の$v_{i-1}$に対して$e^\omega$を掛け合わせ、$v_i$に関する項を足し合わせればよい
- この計算量の増大が、高々$O(n)$として表現できることから、計算コストを削減できる


## Time-mixing block



### Time-mixing blockの動作内容

RWKVの時刻$t$における各ベクトル・パラメータであるReceptanceベクトル($r_t$)、Key($k_t$)、Value($v_t$)は、次の式で表すことができる

$$r_t=W_r\cdot(\mu_rx_t+(1−\mu_r)x_{t−1}) \\
k_t=W_k\cdot(\mu_kx_t+(1−\mu_k)x_{t−1}) \\
v_t=W_v\cdot(\mu_vx_t+(1−\mu_v)x_{t−1})$$

これらの値は時刻$t$での更新割合を制御するパラメータであり、0と1の間の値をとる
- $\mu_r$​が大きいほど新しい入力$x_t$​の影響が強く、小さいほど過去の状態$x_{t−1}$​の影響が強くなる

$mu_r$倍の新しい入力(x_t)と$1-mu_r$倍の和であることから、新しい入力と過去の状態との間で線形補間を行う式
- 従って、これを繰り返すことで再帰的に更新できる

これらを使った演算の詳細は次の通りであり、An Attention Free Transformer (AFT）と類似している

$$wk_vt=\frac{\sum_{i=1}^{t−1}e^{−(t−1−i)w+k_i}v_i+e^{u+k_t}v_t}{\sum_{i=1}^{t−1}e^{−(t−1−i)w+k_i}+e^{u+k_t}}$$
$$o_t=W_o\cdot(\sigma(r_t)\odot wk_vt)$$

$wkv_t$​は、$Attn(Q,K,V)$と同様の役割を担っているが、$Q，K，V$の各要素はスカラーのため計算コストが小さい

- 直感的には、時間$t$が増加するにつれてベクトル$o_t$​は長い履歴に依存することを示しているといえる
- ターゲットポジション$t$に対して、RWKVは位置間隔$[1, t]$での加重平均とレセプタンス$sigmoid(R)$と乗算している
  - 与えられたタイムステップ内で乗算され、異なるタイムステップで合計される
- 標準的なトランスフォーマーは全てのトークンのペア間でアテンションを計算するが、AFTは過去の時間ステップ全てにわたる加算の形でアテンションを計算するため、計算とメモリ利用効率が向上している

### 時間減衰率(Time-Weighting)


RWKVはAFTに倣い、$\omega_{t,i}$​をチャネルごとの時間減衰ベクトルとして定義している

$$\omega_{t,i} = -t(t-i)\omega$$

このモデルではTime-MixingおよびTime-Weightingというパラメータを導入している
- Time-Weightingを距離によるAttentionと呼ぶ

Wはヘッド数、block_size(0～現在の時刻)、block_sizeの3次元で構成され、attに$\mathbb{W}$を要素ごとかけた値をdropoutしている

$\mathbb{W}$は比較的小さいパラメータであり、このようなパラメ―タで離れた単語の依存関係を学習可能とする

これは、異なる距離のトークン($W[:, block_size, block_size]$)が現在の単語attに与える影響は異なり、特に文章の後半は長い距離の$\mathbb{W}$を用いる必要があるが、最初は履歴が小さく必要がない

- SelfAttentionはそのような考慮がない

実際には$\mathbb{W}$は巡回行列でありバイアスを加算する必要があり、Time Weightingの導入によりPositionalEncodingが不要になる

### Time-mixing blockの実装

下記コードの関数`RWKV_TimeMix`の概要は次の通り
- 時刻$t$における入力の更新割合$µ_r$​はself.time_mixで表される
- また時刻$t−1$の入力xはnn.ZeroPad2dを用いて表されている
- これらを用いると入力$x$は、`x = x * self.time_mix + self.time_shift(x) * (1 - self.time_mix)`と実装され、時間的にシフトした部分すなわち過去の情報と、シフトしていない部分すなわち現在の情報を適切な比率で混合した値へと変換している


## Channel-mixing block

channel-mixing blockは次の式で表される

$$r_t=W_r\cdot(\mu_rx_t+(1−\mu_r)x_{t−1})$$
$$k_t=W_k\cdot(\mu_kx_t+(1−\mu_k)x_{t−1})$$
$$o_t=\sigma(r_t)\odot(W_v\cdot max(k_t,0)^2)$$

Time-mixing blockは異なる時間ステップのトークン間の相互作用を管理するのに対して、Channel-mixing blockは同じ時間ステップ内の異なるチャンネル（または特徴）間の相互作用を管理する

Channel-mixing blockは、全結合層や畳み込み層と同様の働きを持つ
- $\sigma$はsquared ReLUを使用し、忘却ゲートの役割を果たす

Channel-mixing blockの実装は、Time-mixing blockが理解できれば問題なく理解であろう

# RWKVの特徴

RWKVは学習時time-parallel mode (時間パラレルモード)が使用され、推論時（デコード時）はtime-sequential mode (時間シーケンシャルモード)が使用される

<img src="http://class.west.sd.keio.ac.jp/dataai/text/rwkv.png" width=500>


## 時間パラレルモード

学習時のモード

時間パラレルモードは言葉の通り、時刻に関連する演算を並列して行う

Time-mixing blockで説明したように新しい入力$x_t$と過去の状態との間で線形補間を行う
- 結果として、各タイムステップの計算が他のタイムステップの計算と独立に実行できる
- RNNとは異なり並列処理が可能

## 時間シーケンシャルモード

推論時のモード

学習時とは異なり、推論時にはRNNのような順次的なデコーディングを行う

- このときRWKVはRNNと同様の構造を活用して動作する
  - これを時間シーケンシャルモードと呼ぶ
  - 各ステップの出力が次のステップの入力として用いられる
  - RNN同様、は出力トークンを一度に1つずつ生成し、あるトークンの生成は前のすべてのトークンの生成が完了した後に行わる

RWKVはシーケンスの長さに関係なく一定の速度とメモリフットプリントを維持しする
- 長いシーケンスを効率的に処理できる
- 一方で、アテンションメカニズムを使用しているため、シーケンスの長さに比例してキャッシュの使用量が増加する

以上からRWKVではTransfromerでは実現できなかった効率的なデコーディングが可能となる

# ChatRWKV-RWKV4-Worldを試す

transformerレベルの性能を持つRNNである、[RWKV](https://github.com/BlinkDL/RWKV-LM)を利用して、実際にRWKVを試す
- ここでは、[ChatRWKV](https://github.com/BlinkDL/RWKV-LM) RWKVの推論用モデルを利用する

利用にあたって、学習済みモデルやドライブのマウント場所など、必要な設定を行う

In [1]:
save_models_to_drive = True # ドライブをマウントしてモデルを保存、利用する
drive_mount = '/content/drive' # ドライブをマウントする場所を指定する
model_dir = 'rwkv-4-world-model' # モデルを保存する場所を指定する

import os
if save_models_to_drive:
    from google.colab import drive
    drive.mount(drive_mount)
    model_dir_path = f"{drive_mount}/MyDrive/{model_dir}" if save_models_to_drive else f"/content/{model_dir}"
else:
    model_dir_path = "/content"

os.makedirs(f"{model_dir_path}", exist_ok=True)

print(f"Saving models to {model_dir_path}")

Mounted at /content/drive
Saving models to /content/drive/MyDrive/rwkv-4-world-model


In [2]:
!git clone https://github.com/BlinkDL/ChatRWKV

Cloning into 'ChatRWKV'...
remote: Enumerating objects: 1977, done.
remote: Counting objects: 100% (884/884), done.
remote: Compressing objects: 100% (320/320), done.
remote: Total 1977 (delta 703), reused 632 (delta 547), pack-reused 1093 (from 1)
Receiving objects: 100% (1977/1977), 30.55 MiB | 14.01 MiB/s, done.
Resolving deltas: 100% (1120/1120), done.


In [3]:
!pip install ninja tokenizers rwkv prompt_toolkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 406.2/406.2 kB 27.5 MB/s eta 0:00:00


## モデルの選択とダウンロード



最初に `model_dir` から `model_file` を検索する。
- 有効なパスでない場合、huggingfaceから `RWKV-4-World` モデルのダウンロードを試みる

ここでは、日本語を利用するが、その他どのようなモデルがあるかは[repo](https://huggingface.co/BlinkDL/rwkv-4-world/)を参照すること
- RWKV-4-World-JPNtuned-7B: `RWKV-4-World-JPNtuned-7B-v1-20230718-ctx4096.pth`
- RWKV-4-World-CHNtuned-7B: `RWKV-4-World-CHNtuned-7B-v1-20230709-ctx4096.pth`

ダウンロードには、20分弱必要となる

In [4]:
import urllib

model_file = "RWKV-4-World-JPNtuned-7B-v1-20230718-ctx4096.pth" # モデルを指定する(ここでは日本語モデルとする)

model_path = f"{model_dir_path}/{model_file}"
if not os.path.exists(model_path):
    model_repo = f"https://huggingface.co/BlinkDL/rwkv-4-world/resolve/main"
    model_url = f"{model_repo}/{urllib.parse.quote_plus(model_file)}"
    try:
        print(f"Downloading '{model_file}' from {model_url} this may take a while")
        urllib.request.urlretrieve(model_url, model_path)
        print(f"Using {model_path} as base")
    except Exception as e:
        print(f"Model '{model_file}' doesn't exist")
        raise Exception
else:
    print(f"Using {model_path} as base")

Using /content/drive/MyDrive/rwkv-4-world-model/RWKV-4-World-JPNtuned-7B-v1-20230718-ctx4096.pth as base


## モデルのロード

Strategyの例:
- `cpu fp32`
- `cuda:0 fp16 -> cuda:1 fp16`
- `cuda fp16i8 *10 -> cuda fp16`
- `cuda fp16i8`
- `cuda fp16i8 -> cpu fp32 *10`
- `cuda fp16i8 *10+`

RWKV_JIT_ONの例:
- `1` でJITコンパイラ有効(有効にするべき)
- `0` でJITコンパイラ無効

JITコンパイラを利用するため、 torch 1.13+ が必要

RWKV_CUDA_ONの例:
- `1` でCUDA有効(GPU使用を推奨)
- `0` でCUDA無効

このモデルを利用するには、ハイメモリが必要となる

In [5]:
import os, copy, types, gc, sys
import numpy as np
import torch
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cuda.matmul.allow_tf32 = True

strategy = 'cuda fp16'
RWKV_JIT_ON = '1'
RWKV_CUDA_ON = '1'
os.environ["RWKV_JIT_ON"] = '1'
os.environ["RWKV_CUDA_ON"] = '1'

## モデルのセットアップ


CHAT_LANGは日本語でよい

PROMPT_FILEは、次の違いがある
- -1.pyとする場合は、UserとBotでQ&A形式のプロンプトとして運用する
- -2.pyとする場合は、BobとAliceのチャットのような、会話プロンプトとして運用する

ここでは、会話プロンプトとしている

パラメータ説明は次の通り
- GEN_TEMP : 温度
- GEN_TOP_P : top-P
- GEN_alpha_presence : 同じトークンの繰り返しを減らすため出力トークンに課すペナルティ (-2〜2)
- GEN_penalty_decay : 同じテキスト出力をへらすため出力トークンに課すペナルティ (-2〜2)
- GEN_penalty_decay : 新しいトークンがこれまでのテキストに表示されているかどうかに基づいてペナルティを課し、モデルが新しいトピックについて話す可能性を高める

詳細は、[API](https://platform.openai.com/docs/api-reference/completions/create)仕様を参照のこと

また、CHUNK_LENは、VRAMを節約するために入力をチャンクに分割する際のチャンクサイズを指定する

より良いチャットとQAのためには、tempの値、およびtop-pの値を減らし、繰り返しのペナルティを増やすとよい
- この点について、https://platform.openai.com/docs/api-reference/parameter-details の解説を参照するとよい
- Q&A の精度を高る、特に多様性を減らすために、top_p を0.5、0.2、0.1 などといった小さな値にするとよいであろう

In [6]:
CHAT_LANG = 'Japanese'
PROMPT_FILE = f'/content/ChatRWKV/v2/prompt/default/{CHAT_LANG}-2.py'

CHAT_LEN_SHORT = 40
CHAT_LEN_LONG = 150
FREE_GEN_LEN = 256

GEN_TEMP = 1.2
GEN_TOP_P = 0.5
GEN_alpha_presence = 0.4
GEN_alpha_frequency = 0.4
GEN_penalty_decay = 0.996
AVOID_REPEAT = '，：？！'

CHUNK_LEN = 256

print(f'\n{CHAT_LANG} - {strategy} - {PROMPT_FILE}')
from rwkv.model import RWKV
from rwkv.utils import PIPELINE

def load_prompt(PROMPT_FILE):
    variables = {}
    with open(PROMPT_FILE, 'rb') as file:
        exec(compile(file.read(), PROMPT_FILE, 'exec'), variables)
    user, bot, interface, init_prompt = variables['user'], variables['bot'], variables['interface'], variables['init_prompt']
    init_prompt = init_prompt.strip().split('\n')
    for c in range(len(init_prompt)):
        init_prompt[c] = init_prompt[c].strip().strip('\u3000').strip('\r')
    init_prompt = '\n' + ('\n'.join(init_prompt)).strip() + '\n\n'
    return user, bot, interface, init_prompt

# Load Model

print(f'Loading model - {model_path}')
model = RWKV(model=model_path, strategy=strategy)

pipeline = PIPELINE(model, "rwkv_vocab_v20230424")
END_OF_TEXT = 0
END_OF_LINE = 11

# pipeline = PIPELINE(model, "cl100k_base")
# END_OF_TEXT = 100257
# END_OF_LINE = 198

model_tokens = []
model_state = None

AVOID_REPEAT_TOKENS = []
for i in AVOID_REPEAT:
    dd = pipeline.encode(i)
    assert len(dd) == 1
    AVOID_REPEAT_TOKENS += dd

########################################################################################################

def run_rnn(tokens, newline_adj = 0):
    global model_tokens, model_state

    tokens = [int(x) for x in tokens]
    model_tokens += tokens
    # print(f'### model ###\n{tokens}\n[{pipeline.decode(model_tokens)}]')

    while len(tokens) > 0:
        out, model_state = model.forward(tokens[:CHUNK_LEN], model_state)
        tokens = tokens[CHUNK_LEN:]

    out[END_OF_LINE] += newline_adj # adjust \n probability

    if model_tokens[-1] in AVOID_REPEAT_TOKENS:
        out[model_tokens[-1]] = -999999999
    return out

all_state = {}
def save_all_stat(srv, name, last_out):
    n = f'{name}_{srv}'
    all_state[n] = {}
    all_state[n]['out'] = last_out
    all_state[n]['rnn'] = copy.deepcopy(model_state)
    all_state[n]['token'] = copy.deepcopy(model_tokens)

def load_all_stat(srv, name):
    global model_tokens, model_state
    n = f'{name}_{srv}'
    model_state = copy.deepcopy(all_state[n]['rnn'])
    model_tokens = copy.deepcopy(all_state[n]['token'])
    return all_state[n]['out']

# Model only saw '\n\n' as [187, 187] before, but the tokenizer outputs [535] for it at the end
def fix_tokens(tokens):
        return tokens


Japanese - cuda fp16 - /content/ChatRWKV/v2/prompt/default/Japanese-2.py


Using /root/.cache/torch_extensions/py310_cu121 as PyTorch extensions root...
Creating extension directory /root/.cache/torch_extensions/py310_cu121/wkv_cuda...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py310_cu121/wkv_cuda/build.ninja...
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1967: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module wkv_cuda...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module wkv_cuda...


Loading model - /content/drive/MyDrive/rwkv-4-world-model/RWKV-4-World-JPNtuned-7B-v1-20230718-ctx4096.pth
RWKV_JIT_ON 1 RWKV_CUDA_ON 1 RESCALE_LAYER 6

Loading /content/drive/MyDrive/rwkv-4-world-model/RWKV-4-World-JPNtuned-7B-v1-20230718-ctx4096.pth ...
Model detected: v4.0
Strategy: (total 32+1=33 layers)
* cuda [float16, float16], store 33 layers
0-cuda-float16-float16 1-cuda-float16-float16 2-cuda-float16-float16 3-cuda-float16-float16 4-cuda-float16-float16 5-cuda-float16-float16 6-cuda-float16-float16 7-cuda-float16-float16 8-cuda-float16-float16 9-cuda-float16-float16 10-cuda-float16-float16 11-cuda-float16-float16 12-cuda-float16-float16 13-cuda-float16-float16 14-cuda-float16-float16 15-cuda-float16-float16 16-cuda-float16-float16 17-cuda-float16-float16 18-cuda-float16-float16 19-cuda-float16-float16 20-cuda-float16-float16 21-cuda-float16-float16 22-cuda-float16-float16 23-cuda-float16-float16 24-cuda-float16-float16 25-cuda-float16-float16 26-cuda-float16-float16 27-cuda-f

## デモの出力

In [7]:
# Run inference
print(f'\nRun prompt...')

user, bot, interface, init_prompt = load_prompt(PROMPT_FILE)
out = run_rnn(fix_tokens(pipeline.encode(init_prompt)))
save_all_stat('', 'chat_init', out)
gc.collect()
torch.cuda.empty_cache()

srv_list = ['dummy_server']
for s in srv_list:
    save_all_stat(s, 'chat', out)

def reply_msg(msg):
    print(f'{bot}{interface} {msg}\n')

def on_message(message):
    global model_tokens, model_state, user, bot, interface, init_prompt

    srv = 'dummy_server'

    msg = message.replace('\\n','\n').strip()

    x_temp = GEN_TEMP
    x_top_p = GEN_TOP_P
    if ("-temp=" in msg):
        x_temp = float(msg.split("-temp=")[1].split(" ")[0])
        msg = msg.replace("-temp="+f'{x_temp:g}', "")
        # print(f"temp: {x_temp}")
    if ("-top_p=" in msg):
        x_top_p = float(msg.split("-top_p=")[1].split(" ")[0])
        msg = msg.replace("-top_p="+f'{x_top_p:g}', "")
        # print(f"top_p: {x_top_p}")
    if x_temp <= 0.2:
        x_temp = 0.2
    if x_temp >= 5:
        x_temp = 5
    if x_top_p <= 0:
        x_top_p = 0
    msg = msg.strip()

    if msg == '+reset':
        out = load_all_stat('', 'chat_init')
        save_all_stat(srv, 'chat', out)
        reply_msg("Chat reset.")
        return

    # use '+prompt {path}' to load a new prompt
    elif msg[:8].lower() == '+prompt ':
        print("Loading prompt...")
        try:
            PROMPT_FILE = msg[8:].strip()
            user, bot, interface, init_prompt = load_prompt(PROMPT_FILE)
            out = run_rnn(fix_tokens(pipeline.encode(init_prompt)))
            save_all_stat(srv, 'chat', out)
            print("Prompt set up.")
            gc.collect()
            torch.cuda.empty_cache()
        except:
            print("Path error.")

    elif msg[:5].lower() == '+gen ' or msg[:3].lower() == '+i ' or msg[:4].lower() == '+qa ' or msg[:4].lower() == '+qq ' or msg.lower() == '+++' or msg.lower() == '++':

        if msg[:5].lower() == '+gen ':
            new = '\n' + msg[5:].strip()
            # print(f'### prompt ###\n[{new}]')
            model_state = None
            model_tokens = []
            out = run_rnn(pipeline.encode(new))
            save_all_stat(srv, 'gen_0', out)

        elif msg[:3].lower() == '+i ':
            msg = msg[3:].strip().replace('\r\n','\n').replace('\n\n','\n')
            new = f'''
Below is an instruction that describes a task. Write a response that appropriately completes the request.

# Instruction:
{msg}

# Response:
'''
            # print(f'### prompt ###\n[{new}]')
            model_state = None
            model_tokens = []
            out = run_rnn(pipeline.encode(new))
            save_all_stat(srv, 'gen_0', out)

        elif msg[:4].lower() == '+qq ':
            new = '\nQ: ' + msg[4:].strip() + '\nA:'
            # print(f'### prompt ###\n[{new}]')
            model_state = None
            model_tokens = []
            out = run_rnn(pipeline.encode(new))
            save_all_stat(srv, 'gen_0', out)

        elif msg[:4].lower() == '+qa ':
            out = load_all_stat('', 'chat_init')

            real_msg = msg[4:].strip()
            new = f"{user}{interface} {real_msg}\n\n{bot}{interface}"
            # print(f'### qa ###\n[{new}]')

            out = run_rnn(pipeline.encode(new))
            save_all_stat(srv, 'gen_0', out)

        elif msg.lower() == '+++':
            try:
                out = load_all_stat(srv, 'gen_1')
                save_all_stat(srv, 'gen_0', out)
            except:
                return

        elif msg.lower() == '++':
            try:
                out = load_all_stat(srv, 'gen_0')
            except:
                return

        begin = len(model_tokens)
        out_last = begin
        occurrence = {}
        for i in range(FREE_GEN_LEN+100):
            for n in occurrence:
                out[n] -= (GEN_alpha_presence + occurrence[n] * GEN_alpha_frequency)
            token = pipeline.sample_logits(
                out,
                temperature=x_temp,
                top_p=x_top_p,
            )
            if token == END_OF_TEXT:
                break
            for xxx in occurrence:
                occurrence[xxx] *= GEN_penalty_decay
            if token not in occurrence:
                occurrence[token] = 1
            else:
                occurrence[token] += 1

            if msg[:4].lower() == '+qa ':# or msg[:4].lower() == '+qq ':
                out = run_rnn([token], newline_adj=-2)
            else:
                out = run_rnn([token])

            xxx = pipeline.decode(model_tokens[out_last:])
            if '\ufffd' not in xxx: # avoid utf-8 display issues
                print(xxx, end='', flush=True)
                out_last = begin + i + 1
                if i >= FREE_GEN_LEN:
                    break
        print('\n')
        # send_msg = pipeline.decode(model_tokens[begin:]).strip()
        # print(f'### send ###\n[{send_msg}]')
        # reply_msg(send_msg)
        save_all_stat(srv, 'gen_1', out)

    else:
        if msg.lower() == '+':
            try:
                out = load_all_stat(srv, 'chat_pre')
            except:
                return
        else:
            out = load_all_stat(srv, 'chat')
            msg = msg.strip().replace('\r\n','\n').replace('\n\n','\n')
            new = f"{user}{interface} {msg}\n\n{bot}{interface}"
            # print(f'### add ###\n[{new}]')
            out = run_rnn(pipeline.encode(new), newline_adj=-999999999)
            save_all_stat(srv, 'chat_pre', out)

        begin = len(model_tokens)
        out_last = begin
        print(f'{bot}{interface}', end='', flush=True)
        occurrence = {}
        for i in range(999):
            if i <= 0:
                newline_adj = -999999999
            elif i <= CHAT_LEN_SHORT:
                newline_adj = (i - CHAT_LEN_SHORT) / 10
            elif i <= CHAT_LEN_LONG:
                newline_adj = 0
            else:
                newline_adj = min(3, (i - CHAT_LEN_LONG) * 0.25) # MUST END THE GENERATION

            for n in occurrence:
                out[n] -= (GEN_alpha_presence + occurrence[n] * GEN_alpha_frequency)
            token = pipeline.sample_logits(
                out,
                temperature=x_temp,
                top_p=x_top_p,
            )
            # if token == END_OF_TEXT:
            #     break
            for xxx in occurrence:
                occurrence[xxx] *= GEN_penalty_decay
            if token not in occurrence:
                occurrence[token] = 1
            else:
                occurrence[token] += 1

            out = run_rnn([token], newline_adj=newline_adj)
            out[END_OF_TEXT] = -999999999  # disable <|endoftext|>

            xxx = pipeline.decode(model_tokens[out_last:])
            if '\ufffd' not in xxx: # avoid utf-8 display issues
                print(xxx, end='', flush=True)
                out_last = begin + i + 1

            send_msg = pipeline.decode(model_tokens[begin:])
            if '\n\n' in send_msg:
                send_msg = send_msg.strip()
                break

            # send_msg = pipeline.decode(model_tokens[begin:]).strip()
            # if send_msg.endswith(f'{user}{interface}'): # warning: needs to fix state too !!!
            #     send_msg = send_msg[:-len(f'{user}{interface}')].strip()
            #     break
            # if send_msg.endswith(f'{bot}{interface}'):
            #     send_msg = send_msg[:-len(f'{bot}{interface}')].strip()
            #     break

        # print(f'{model_tokens}')
        # print(f'[{pipeline.decode(model_tokens)}]')

        # print(f'### send ###\n[{send_msg}]')
        # reply_msg(send_msg)
        save_all_stat(srv, 'chat', out)

########################################################################################################

if CHAT_LANG == 'English':
    HELP_MSG = '''Commands:
say something --> chat with bot. use \\n for new line.
+ --> alternate chat reply
+reset --> reset chat

+gen YOUR PROMPT --> free single-round generation with any prompt. use \\n for new line.
+i YOUR INSTRUCT --> free single-round generation with any instruct. use \\n for new line.
+++ --> continue last free generation (only for +gen / +i)
++ --> retry last free generation (only for +gen / +i)

Now talk with the bot and enjoy. Remember to +reset periodically to clean up the bot's memory. Use RWKV-4 14B (especially https://huggingface.co/BlinkDL/rwkv-4-raven) for best results.
'''
elif CHAT_LANG == 'Chinese':
    HELP_MSG = f'''指令:
直接输入内容 --> 和机器人聊天（建议问机器人问题），用\\n代表换行，必须用 Raven 模型
+ --> 让机器人换个回答
+reset --> 重置对话，请经常使用 +reset 重置机器人记忆

+i 某某指令 --> 问独立的问题（忽略聊天上下文），用\\n代表换行，必须用 Raven 模型
+gen 某某内容 --> 续写内容（忽略聊天上下文），用\\n代表换行，写小说用 testNovel 模型
+++ --> 继续 +gen / +i 的回答
++ --> 换个 +gen / +i 的回答

作者：彭博 请关注我的知乎: https://zhuanlan.zhihu.com/p/603840957
如果喜欢，请看我们的优质护眼灯: https://withablink.taobao.com

中文 Novel 模型，可以试这些续写例子（不适合 Raven 模型）：
+gen “区区
+gen 以下是不朽的科幻史诗长篇巨著，描写细腻，刻画了数百位个性鲜明的英雄和宏大的星际文明战争。\\n第一章
+gen 这是一个修真世界，详细世界设定如下：\\n1.
'''
elif CHAT_LANG == 'Japanese':
    HELP_MSG = f'''コマンド:
直接入力 --> ボットとチャットする．改行には\\nを使用してください．
+ --> ボットに前回のチャットの内容を変更させる．
+reset --> 対話のリセット．メモリをリセットするために，+resetを定期的に実行してください．

+i インストラクトの入力 --> チャットの文脈を無視して独立した質問を行う．改行には\\nを使用してください．
+gen プロンプトの生成 --> チャットの文脈を無視して入力したプロンプトに続く文章を出力する．改行には\\nを使用してください．
+++ --> +gen / +i の出力の回答を続ける．
++ --> +gen / +i の出力の再生成を行う.

ボットとの会話を楽しんでください。また、定期的に+resetして、ボットのメモリをリセットすることを忘れないようにしてください。
'''

print(f'{pipeline.decode(model_tokens)}'.replace(f'\n\n{bot}',f'\n{bot}'), end='')

########################################################################################################





Run prompt...

以下は、Aliceという女の子とその友人Bobの間で行われた会話です。 Aliceはとても賢く、想像力があり、友好的です。 AliceはBobに反対することはなく、AliceはBobに質問するのは苦手です。 AliceはBobに自分のことや自分の意見をたくさん伝えるのが好きです。 AliceはいつもBobに親切で役に立つ、有益なアドバイスをしてくれます。

Bob: こんにちはAlice、調子はどうですか？
Alice: こんにちは！元気ですよ。あたなはどうですか？

Bob: 元気ですよ。君に会えて嬉しいよ。見て、この店ではお茶とジュースが売っているよ。
Alice: 本当ですね。中に入りましょう。大好きなモカラテを飲んでみたいです！

Bob: モカラテって何ですか？
Alice: モカラテはエスプレッソ、ミルク、チョコレート、泡立てたミルクから作られた飲み物です。香りはとても甘いです。

Bob: それは美味しそうですね。今度飲んでみます。しばらく私とおしゃべりしてくれますか？
Alice: もちろん！ご質問やアドバイスがあれば、喜んでお答えします。専門的な知識には自信がありますよ。どうぞよろしくお願いいたします！



## チャット

このセルを実行するとチャットが開始される
- 入力欄にメッセージを入力するとそれに対する返答を得ることができる

次のコマンドが用意されている:
- `+`：代替のチャット応答を取得する
- `+reset`：チャットをリセットする
- `+gen YOUR PROMPT`：任意のプロンプトでの無料(記録しない)の単一ラウンド生成に使用する
- `+i YOUR INSTRUCT`：任意の指示での無料の単一ラウンド生成に使用する
- `+++`：最後の無料生成を続行するために使用する（`+gen` / `+i`のみ）
- `++`：最後の無料生成を再試行するために使用する（`+gen` / `+i`のみ）

RWKVは過去の入力内容を保持し続けるため、定期的にボットのメモリをクリーンアップするために、`+reset`を実行するのを忘れないようにすること


In [ ]:
from prompt_toolkit import prompt
while True:
    msg = input("Bob: ")
    if len(msg.strip()) > 0:
        on_message(msg)
    else:
        print('Error: please say something')

Bob: こんにちは
Alice: こんにちはBob、調子はどうですか？

Bob: 新しいDNNモデルであるRWKVについて学んでいます。なにに注意すればよいですか？
Alice: こんにちはBob、調子はとても良いです。RWKVについて学んでいますか？

Bob: はい。RWKVについて学んでいます。なにに注意すればよいですか？
Alice: はい。RWKVについて学んでいますか？



# RWKVの学習

実際にRWKVを用いた学習を行うことができる。ただし、学習にGoogle Colaboratory ProでA100を利用した場合でも10時間強程度必要であり、T4などでは24時間を超えるため学習が終了しない。高価なA100を利用し、基本的に放置になるのと同時に、それなりの課金が必要となる

従って、授業内容としては適さないと判断しており、ここでは、そのリンクのみ示すとして、実際に実行することは避ける
- また、このリンクについて、動作を完全に保証するものではない点も留意願いたい
- 学習を行うノートブックは、mlsys-text-O-RWKV-Learning.ipynbである


課題1

RWKVはなぜ期待されているのか、自分の言葉で纏めなさい

課題2

その他、注目する技術は次々と登場している

何でもよいので、最新の機械学習に関連する技術について調査を行い、簡単に纏めなさい